In [174]:
import pandas as pd
import os
from faker import Faker

baseDir = '/Users/Sanket/Documents/GitHub/Real-Time-Data-Pipeline'
userDataDir = baseDir + '/output/userData/'
fake = Faker()

In [160]:
if not os.path.exists(userDataDir):
    os.mkdir(userDataDir)
for i in range(1,10001):
    data = {
        'name':fake.name(),
        'id':i,
        'street':fake.street_address(),
        'city':fake.city(),
        'zip':fake.zipcode()
    }
    df = pd.DataFrame([data])
    df.to_json(userDataDir+str(data['id'])+'.json')

In [156]:
userDF = pd.DataFrame()

fileNames = [x for x in os.listdir(userDataDir) if x.endswith('.json')]
for name in fileNames:
    df = pd.read_json(userDataDir+name)
    userDF = userDF.append(other=df,ignore_index=True)
print('Found {} Records In Data Lake '.format(userDF.id.count()))


Found 9389 Records In Data Lake 


In [153]:
import psycopg2 as db

conn_string = "dbname='test' host='localhost' user='postgres' password='postgres'"
try:
    conn = db.connect(conn_string)
    cur = conn.cursor()
    query = "insert into users (name,id,street,city,zip) values(%s,%s,%s,%s,%s)"
    #print(cur.mogrify(query,tuple(userDF.values)))
    cur.executemany(query,userDF.values)
    print(conn.commit())
except Exception as e:
    print(str(e))

duplicate key value violates unique constraint "users_pkey"
DETAIL:  Key (id)=(4581) already exists.



In [166]:
pd.read_sql('select count(*) from users;',conn)

,count
0,0


In [167]:
cur.execute('truncate table users;')
conn.commit()

In [89]:
userDF.id.count()
print('Found {} Records In Data Lake '.format(userDF.id.count()))
print('Inserting {} Records In Staging DB'.format(userDF.id.count()))


Found 28167 Records In Data Lake 
Inserting 28167 Records In Staging DB


In [172]:
#conn.rollback()
userDF[userDF['id']==3721]



,name,id,street,city,zip


In [171]:
userDF.drop(5620,inplace=True)

In [181]:
fileNames = [x for x in os.listdir(userDataDir) if x.endswith('.json')]
for name in fileNames:
    if os.path.exists(name):
        os.remove(name)


